In [1]:
from lxml import html, etree
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests, re, json, time

In [2]:
zillow_url = 'https://www.zillow.com/orlando-fl/rentals/2-_beds/2.0-_baths/?searchQueryState={"pagination":{},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-81.531103405855,"east":-81.1294157837847,"south":28.323397464539266,"north":28.565502210214134},"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":false,"filterState":{"beds":{"min":2},"baths":{"min":2},"fore":{"value":false},"ah":{"value":true},"auc":{"value":false},"nc":{"value":false},"fr":{"value":true},"fsbo":{"value":false},"cmsn":{"value":false},"fsba":{"value":false},"tow":{"value":false},"apco":{"value":false},"apa":{"value":false},"con":{"value":false}},"isListVisible":true,"mapZoom":12}'

def default_headers():
    headers = {}
    headers['user-agent'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
    headers['accept'] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
    headers['accept-encoding'] = 'gzip, deflate, br'
    headers['accept-language'] = 'en-US,en;q=0.9'
    headers['cache-control'] = 'max-age=0'
    headers['sec-ch-ua'] = '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"'
    headers['sec-ch-ua-mobile'] = '?0'
    headers['sec-ch-ua-platform'] = '"Windows"'
    headers['sec-fetch-dest'] = 'document'
    headers['sec-fetch-mode'] = 'navigate'
    headers['sec-fetch-site'] = 'same-origin'
    headers['sec-fetch-user'] = '?1'
    headers['upgrade-insecure-requests'] = '1'
    return headers

In [ ]:
#def default_headers():
    headers = {}
    headers['user-agent'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
    headers['accept'] = "*/*"
    headers['accept-encoding'] = 'gzip, deflate, br'
    headers['accept-language'] = 'en-US,en;q=0.9'
    headers['sec-ch-ua'] = '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"'
    headers['sec-ch-ua-mobile'] = '?0'
    headers['sec-ch-ua-platform'] = '"Windows"'
    headers['sec-fetch-dest'] = 'empty'
    headers['sec-fetch-mode'] = 'cors'
    headers['sec-fetch-site'] = 'same-origin'
    return headers

https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"mapBounds":{"west":-81.98262205821744,"east":-81.13804808360807,"south":27.80081193406963,"north":28.85368975780137},"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}},"isListVisible":true}&wants={"cat1":["listResults","mapResults"]}&requestId=9

In [3]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-82.36371031505338,"east":-80.75695982677213,"south":27.80081193406963,"north":28.85368975780137},"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-82.36371031505338,"east":-80.75695982677213,"south":27.80081193406963,"north":28.85368975780137},"mapZoom":12,"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results.extend(zillow_json['cat1']['searchResults']['listResults'])

In [68]:
#Alex, the list of zillow search results are stored in "zillow_results", a list of dicts containing the rental property info
#it looks like the 'providerListingId' key might be unique enough to use as an identifier to see which ones you have already looked at before
#but 'address' might be more user-friendly
zillow_results[2]


{'zpid': '2066382103',
 'id': '2066382103',
 'providerListingId': '3q4jqbce0u1yw',
 'imgSrc': 'https://photos.zillowstatic.com/fp/05b739e8e9af57a4fe847383fce9be7c-p_e.jpg',
 'hasImage': True,
 'detailUrl': 'https://www.zillow.com/homedetails/2520-S-Conway-Rd-FF7185F4E-Orlando-FL-32812/2066382103_zpid/',
 'statusType': 'FOR_RENT',
 'statusText': 'Apartment for rent',
 'countryCurrency': '$',
 'price': '$1,579+/mo',
 'unformattedPrice': 1579,
 'address': 'Conway Forest, 2520 S Conway Rd, Orlando, FL 32812',
 'addressStreet': '2520 S Conway Rd #FF7185F4E',
 'addressCity': 'Orlando',
 'addressState': 'FL',
 'addressZipcode': '32812',
 'isUndisclosedAddress': False,
 'beds': 2,
 'baths': 1.0,
 'area': 910,
 'latLong': {'latitude': 28.514633, 'longitude': -81.331276},
 'isZillowOwned': False,
 'variableData': {'type': 'TIME_ON_INFO',
  'text': None,
  'data': {'isFresh': False}},
 'badgeInfo': None,
 'hdpData': {'homeInfo': {'zpid': 2066382103,
   'streetAddress': '2520 S Conway Rd #FF7185F4

In [90]:
i = 5
zil_id = zillow_results[i]['providerListingId']
zil_price =zillow_results[i]['unformattedPrice']
zil_address = zillow_results[i]['address']
zil_beds = zillow_results[i]['beds']
zil_baths = zillow_results[i]['baths']
zil_avail = zillow_results[i]['availabilityDate']
zil_status = zillow_results[i]['statusText']

unit = [zil_id, zil_address, zil_price, zil_beds, zil_baths, zil_avail]
print(f"ID: '{zil_id}' \n Price: '{zil_price}' \n Address: '{zil_address}' \n Beds: '{zil_beds}' \n Baths: '{zil_baths}' \n Available: '{zil_avail}' \n Status: '{zil_status}'")
unit

ID: '59vsreb9qscdz' 
 Price: '1739' 
 Address: 'Aria Beach Apartments, 3211 S Semoran Blvd #2, Orlando, FL 32822' 
 Beds: '2' 
 Baths: '1.5' 
 Available: '2022-07-01 00:00:00' 
 Status: 'Apartment for rent'


['59vsreb9qscdz',
 'Aria Beach Apartments, 3211 S Semoran Blvd #2, Orlando, FL 32822',
 1739,
 2,
 1.5,
 '2022-07-01 00:00:00']

In [44]:
length = len(zillow_results) +1
length

30

In [93]:
rentals = []
i = 0

for i in zillow_results:
    unit = []
    zil_id = zillow_results[i]['providerListingId']
    zil_price = zillow_results[i]['unformattedPrice']
    zil_address = zillow_results[i]['address']
    zil_beds = zillow_results[i]['beds']
    zil_baths = zillow_results[i]['baths']
    zil_avail = zillow_results[i]['availabilityDate']
    zil_status = zillow_results[i]['statusText']
    # write the variables to UNIT list
    unit = [zil_id, zil_address, zil_price, zil_beds, zil_baths, zil_avail]
    #Put that list into rentals list, for a list of lists to be used in DataFrame later
    rentals.append(unit)
    i += 1
    




TypeError: list indices must be integers or slices, not dict

In [74]:
zillow_results

[{'zpid': '2111009805',
  'id': '2111009805',
  'providerListingId': '11w3zvtcph0be',
  'imgSrc': 'https://photos.zillowstatic.com/fp/660727fe1777cd4f203f5b315a8897b7-p_e.jpg',
  'hasImage': True,
  'detailUrl': 'https://www.zillow.com/homedetails/5501-Conroy-Rd-APT-2-Orlando-FL-32811/2111009805_zpid/',
  'statusType': 'FOR_RENT',
  'statusText': 'Apartment for rent',
  'countryCurrency': '$',
  'price': '$1,559/mo',
  'unformattedPrice': 1559,
  'address': 'Bella Casa Apartments, 5501 Conroy Rd APT 2, Orlando, FL 32811',
  'addressStreet': '5501 Conroy Rd APT 2',
  'addressCity': 'Orlando',
  'addressState': 'FL',
  'addressZipcode': '32811',
  'isUndisclosedAddress': False,
  'beds': 2,
  'baths': 1.5,
  'area': 930,
  'latLong': {'latitude': 28.49404, 'longitude': -81.45837},
  'isZillowOwned': False,
  'variableData': {'type': 'TIME_ON_INFO',
   'text': None,
   'data': {'isFresh': False}},
  'badgeInfo': None,
  'hdpData': {'homeInfo': {'zpid': 2111009805,
    'streetAddress': '55

In [6]:
rentals = []

for result in zillow_results:
    unit = []
    zil_id = result['providerListingId']
    zil_price = result['unformattedPrice']
    zil_address = result['address']
    zil_beds = result['beds']
    zil_baths = result['baths']
    zil_avail = result['availabilityDate']
    zil_status = result['statusText']
    # write the variables to UNIT list
    unit = [zil_id, zil_address, zil_price, zil_beds, zil_baths, zil_avail, zil_status]
    #Put that list into rentals list, for a list of lists to be used in DataFrame later
    rentals.append(unit)
    




KeyError: 'unformattedPrice'

In [81]:
rentals

[]

In [145]:
rentals = []
i = 0

for i in zillow_results[i]:
    unit = []
    zil_id = zillow_results[i]['providerListingId']
    zil_price = ['unformattedPrice']
    zil_address = ['address']
    zil_beds = ['beds']
    zil_baths = ['baths']
    zil_avail = ['availabilityDate']
    zil_status = ['statusText']
    # write the variables to UNIT list
    unit = [zil_id, zil_address, zil_price, zil_beds, zil_baths, zil_avail, zil_status]
    #Put that list into rentals list, for a list of lists to be used in DataFrame later
    rentals.append(unit)

ValueError: invalid literal for int() with base 10: 'zpid'

In [121]:
rentals

[[['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],
  ['baths'],
  ['availabilityDate'],
  ['statusText']],
 [['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],
  ['baths'],
  ['availabilityDate'],
  ['statusText']],
 [['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],
  ['baths'],
  ['availabilityDate'],
  ['statusText']],
 [['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],
  ['baths'],
  ['availabilityDate'],
  ['statusText']],
 [['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],
  ['baths'],
  ['availabilityDate'],
  ['statusText']],
 [['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],
  ['baths'],
  ['availabilityDate'],
  ['statusText']],
 [['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],
  ['baths'],
  ['availabilityDate'],
  ['statusText']],
 [['providerListingId'],
  ['address'],
  ['unformattedPrice'],
  ['beds'],


In [130]:
rentals = []
i = 0

while i < int(length):
    unit = []
    zil_id = zillow_results[i]
    #unit.append(zil_id)
    #zil_price = zillow_results[i]['unformattedPrice']
    #unit.append(zil_price)
    #zil_address = zillow_results[i]['address']
    #unit.append(zil_address)
    #zil_beds = zillow_results[i]['beds']
    #zil_baths = zillow_results[i]['baths']
    #zil_avail = zillow_results[i]['availabilityDate']
   # zil_status = zillow_results[i]['statusText']
    # write the variables to UNIT list
    #unit = [zil_id, zil_address, zil_price, zil_beds, zil_baths, zil_avail, zil_status]
    #Put that list into rentals list, for a list of lists to be used in DataFrame later
    rentals.append(unit)
    i = i + 1

IndexError: list index out of range

In [ ]:
for x in zillow_results:
    print(zillow_results[0])

In [137]:
rentals = []

for result in zillow_results:
    unit = []
    zil_id = result['providerListingId']
    zil_price = result['unformattedPrice']
    zil_address = result['address']
    zil_beds = result['beds']
    zil_baths = result['baths']
    zil_avail = result['availabilityDate']
    rentals.append(unit)



IndexError: list assignment index out of range

In [147]:
import operator, functools
import pandas as pd

In [139]:
get_value = operator.itemgetter('unformattedPrice')
list_of_values = functools.partial(map, get_value)
print(list(list_of_values(zillow_results)))

KeyError: 'unformattedPrice'

In [144]:
zillow_results[0]['price']

'$1,559/mo'

In [153]:
zillow_url = 'https://www.zillow.com/orlando-fl/rentals/2-_beds/2.0-_baths/?searchQueryState={"pagination":{},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-81.531103405855,"east":-81.1294157837847,"south":28.323397464539266,"north":28.565502210214134},"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":false,"filterState":{"beds":{"min":2},"baths":{"min":2},"fore":{"value":false},"ah":{"value":true},"auc":{"value":false},"nc":{"value":false},"fr":{"value":true},"fsbo":{"value":false},"cmsn":{"value":false},"fsba":{"value":false},"tow":{"value":false},"apco":{"value":false},"apa":{"value":false},"con":{"value":false}},"isListVisible":true,"mapZoom":12}'

def default_headers():
    headers = {}
    headers['user-agent'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
    headers['accept'] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
    headers['accept-encoding'] = 'gzip, deflate, br'
    headers['accept-language'] = 'en-US,en;q=0.9'
    headers['cache-control'] = 'max-age=0'
    headers['sec-ch-ua'] = '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"'
    headers['sec-ch-ua-mobile'] = '?0'
    headers['sec-ch-ua-platform'] = '"Windows"'
    headers['sec-fetch-dest'] = 'document'
    headers['sec-fetch-mode'] = 'navigate'
    headers['sec-fetch-site'] = 'same-origin'
    headers['sec-fetch-user'] = '?1'
    headers['upgrade-insecure-requests'] = '1'
    return headers

In [ ]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-82.36371031505338,"east":-80.75695982677213,"south":27.80081193406963,"north":28.85368975780137},"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-82.36371031505338,"east":-80.75695982677213,"south":27.80081193406963,"north":28.85368975780137},"mapZoom":12,"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results.extend(zillow_json['cat1']['searchResults']['listResults'])

In [148]:
df = pd.DataFrame(zillow_results)
df





,zpid,id,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,price,...,hasAdditionalAttributions,isFeaturedListing,availabilityDate,list,relaxed,lotId,units,buildingName,isBuilding,canSaveBuilding
0,2111009805,2111009805,11w3zvtcph0be,https://photos.zillowstatic.com/fp/660727fe177...,True,https://www.zillow.com/homedetails/5501-Conroy...,FOR_RENT,Apartment for rent,$,"$1,559/mo",...,False,True,2022-08-12 00:00:00,True,False,NaN,NaN,NaN,NaN,NaN
1,2066521577,2066521577,3k29nqs8s2g8,https://photos.zillowstatic.com/fp/d80461de01b...,True,https://www.zillow.com/homedetails/4700-Cason-...,FOR_RENT,Apartment for rent,$,"$1,799/mo",...,False,True,2022-07-15 00:00:00,True,False,NaN,NaN,NaN,NaN,NaN
2,2066382103,2066382103,3q4jqbce0u1yw,https://photos.zillowstatic.com/fp/05b739e8e9a...,True,https://www.zillow.com/homedetails/2520-S-Conw...,FOR_RENT,Apartment for rent,$,"$1,579+/mo",...,False,True,None,True,False,NaN,NaN,NaN,NaN,NaN
3,2063482162,2063482162,2ku6mch0x1d7d,https://photos.zillowstatic.com/fp/9a91134887e...,True,https://www.zillow.com/homedetails/4618-Middle...,FOR_RENT,Apartment for rent,$,"$1,669/mo",...,False,True,2022-06-24 00:00:00,True,False,NaN,NaN,NaN,NaN,NaN
4,28.505415--81.30803,28.505415--81.30803,19mfzgqe2tuty,https://photos.zillowstatic.com/fp/c11f2043389...,True,/b/harbor-beach-orlando-fl-5XhxzG/,FOR_RENT,Harbor Beach,NaN,NaN,...,NaN,True,NaN,True,False,1.001423e+09,"[{'price': '$1,549+', 'beds': '2'}]",Harbor Beach,True,True
5,2080162606,2080162606,59vsreb9qscdz,https://photos.zillowstatic.com/fp/8da1cde60e2...,True,https://www.zillow.com/homedetails/3211-S-Semo...,FOR_RENT,Apartment for rent,$,"$1,739/mo",...,False,True,2022-07-01 00:00:00,True,False,NaN,NaN,NaN,NaN,NaN
6,28.55233--81.275665,28.55233--81.275665,3uggj8sen7b8u,https://photos.zillowstatic.com/fp/a9a060f23a1...,True,/b/the-woodlands-apartments-orlando-fl-5Xhy4q/,FOR_RENT,The Woodlands Apartments,NaN,NaN,...,NaN,True,NaN,True,False,1.001424e+09,"[{'price': '$1,610+', 'beds': '2'}]",The Woodlands Apartments,True,True
7,2080432855,2080432855,34kcj0rh3157n,https://photos.zillowstatic.com/fp/fc7659ec4b0...,True,https://www.zillow.com/homedetails/3312-S-Semo...,FOR_RENT,Apartment for rent,$,"$1,579/mo",...,False,True,2022-06-10 00:00:00,True,False,NaN,NaN,NaN,NaN,NaN
8,28.591574--81.24292,28.591574--81.24292,3bfyjjs1z0rk5,https://photos.zillowstatic.com/fp/7ffe9eace88...,True,/b/brooke-commons-orlando-fl-5XhxtW/,FOR_RENT,Brooke Commons,NaN,NaN,...,NaN,True,NaN,True,False,1.001423e+09,"[{'price': '$959+', 'beds': '2'}, {'price': '$...",Brooke Commons,True,True
9,46048553,46048553,304skuu4jfvwy,https://photos.zillowstatic.com/fp/622a9b0af28...,True,https://www.zillow.com/homedetails/1627-29th-S...,FOR_RENT,House for rent,$,"$1,799/mo",...,False,True,2022-06-28 00:00:00,True,False,NaN,NaN,NaN,NaN,NaN


In [149]:
df.columns

Index(['zpid', 'id', 'providerListingId', 'imgSrc', 'hasImage', 'detailUrl',
       'statusType', 'statusText', 'countryCurrency', 'price',
       'unformattedPrice', 'address', 'addressStreet', 'addressCity',
       'addressState', 'addressZipcode', 'isUndisclosedAddress', 'beds',
       'baths', 'area', 'latLong', 'isZillowOwned', 'variableData',
       'badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner',
       'isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate',
       'shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec',
       'hasAdditionalAttributions', 'isFeaturedListing', 'availabilityDate',
       'list', 'relaxed', 'lotId', 'units', 'buildingName', 'isBuilding',
       'canSaveBuilding'],
      dtype='object')

In [151]:
pruned_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed', 'lotId', 'units', 'buildingName', 'isBuilding','canSaveBuilding'])

In [152]:
pruned_df

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,11w3zvtcph0be,Apartment for rent,"$1,559/mo","Bella Casa Apartments, 5501 Conroy Rd APT 2, O...",2.0,1.5,2022-08-12 00:00:00
1,3k29nqs8s2g8,Apartment for rent,"$1,799/mo","Bella Vita Apartments, 4700 Cason Cove Dr #2, ...",2.0,2.0,2022-07-15 00:00:00
2,3q4jqbce0u1yw,Apartment for rent,"$1,579+/mo","Conway Forest, 2520 S Conway Rd, Orlando, FL 3...",2.0,1.0,None
3,2ku6mch0x1d7d,Apartment for rent,"$1,669/mo","Coconut Palms, 4618 Middlebrook Rd #2-1, Orlan...",2.0,1.0,2022-06-24 00:00:00
4,19mfzgqe2tuty,Harbor Beach,NaN,"5757 Five Flags Blvd, Orlando, FL",NaN,NaN,NaN
5,59vsreb9qscdz,Apartment for rent,"$1,739/mo","Aria Beach Apartments, 3211 S Semoran Blvd #2,...",2.0,1.5,2022-07-01 00:00:00
6,3uggj8sen7b8u,The Woodlands Apartments,NaN,"604 Laurel Cove Ct, Orlando, FL",NaN,NaN,NaN
7,34kcj0rh3157n,Apartment for rent,"$1,579/mo","Bellagio, 3312 S Semoran Blvd #2, Orlando, FL ...",2.0,2.0,2022-06-10 00:00:00
8,3bfyjjs1z0rk5,Brooke Commons,NaN,"10237 Eastern Lake Ave, Orlando, FL",NaN,NaN,NaN
9,304skuu4jfvwy,House for rent,"$1,799/mo","1627 29th St, Orlando, FL 32805",3.0,2.0,2022-06-28 00:00:00
